# Chapter 18: Group Testing

In [1]:
import numpy as np
import math
import cvxpy as cp

One attempts to recover an unknown subset $S$ of $\{1,\dots,N\}$ from observations of the type
$$ y_i = \left\{  \begin{matrix} 0 & \mbox{ if } S\cap R_i = \emptyset, \\ 1 & \mbox{ if } S\cap R_i \neq \emptyset, \end{matrix} \right. \quad i = 1,\dots,m, $$
where each $R_i$ is a subset of $\{1,\dots,N\}$.

## Adaptive group testing: binary splitting strategy

In [2]:
# generate a subset of [1:N] with size s
n = 10
N = 2**n
s = 5
S = np.sort(np.random.permutation(N)[:s])

Here, the subsets $R_i$ are chosen adaptively, i.e., they depend on the outcomes of the previous tests. The observation and recovery procedures are thus interwinned.

In [3]:
# recover each index from S one at a time, with n=log_2(N) tests for each of them,
# resulting in m=s*log_2(N) adaptive tests in total
S_rec = []
while np.setdiff1d(S,S_rec).size != 0:
    length_R = N
    R_min = 0
    R_max = N-1
    for i in range(n):
        length_R = int(length_R/2)
        R_left = range(R_min,R_min+length_R)
        R_right = range(R_min+length_R,R_max+1)
        if np.intersect1d(S,np.setdiff1d(R_left,S_rec)).size ==  0:
            R_min = min(R_right)
            R_max = max(R_right)
        else:
            R_min = min(R_left)
            R_max = max(R_left)
    new_idx = R_min
    S_rec.append(new_idx)
print('The original and recovered supports agree:')
print(S)
print(np.asarray(S_rec))

The original and recovered supports agree:
[ 13 342 374 398 852]
[ 13 342 374 398 852]


## Nonadaptive group testing: a deterministically constructed test matrix

Auxiliary function: given integers $d\leq p$, the function below produces the $p\times p^d$ matrix with entries in $\{0,1,\dots,p-1\}$ whose columns indexed by polynomials $f$ of degree $<d$ contains the values $f(0),f(1),\dots,f(p-1)$ modulo $p$.

In [4]:
def value_matrix(p,d):
    M = np.zeros((p,p**d))
    M_1 = np.zeros((p,p))
    for c in range(p):
        M_1[:,c] = c*np.ones(p)
    if d==1:
        M = M_1
    else:
        M_aux =  value_matrix(p,d-1)
        pp = p**(d-1)
        for c in range(p):
            M[:,c*pp:(c+1)*pp] = (c+np.diag(range(p))@M_aux) % p
    return M

The test matrix $A$

In [5]:
p = 13                  # the integer p has to be a prime number
d = 4                   # the integer d must be smaller than p
N = p**d                # the number of columns of the test matrix A
m = p**2                # the number of tests, identified with rows of A
A = np.zeros((m,N))
M = value_matrix(p,d)
for j in range(p-1):
    A[j*p:(j+1)*p,:] = 1*(M==j)

Verification that the matrix $A$ has coherence $ \mu = \frac{d-1}{p} $, so that $A$ is $s$-disjoint whenever $s<\frac{p}{d-1}$.

In [6]:
Gram = A.T @ A
coh = np.max(abs(Gram-np.diag(np.diag(Gram))))/p
print('The coherence of A is {:.3f}, which equals (d-1)/p={:.3f}'.format(coh,(d-1)/p))

The coherence of A is 0.231, which equals (d-1)/p=0.231


## Nonadaptive group testing: recovery via a linear feasibility program

In [7]:
# generate a subset of [1:N] with size s
s = math.ceil( p/(d-1)) - 1
S = np.sort(np.random.permutation(N)[:s])
x = np.zeros(N)
x[S] = np.ones(s)
# produce the vector of test outcomes
y = 1*(A@x > 0)
# solve the linear feasibility program
I0 = np.where( y==0 )
I1 = np.where( y==1 )
z = cp.Variable(N,nonneg=True)
objective = cp.Minimize(1)
constraints = [A[I0[0],:]@z==0]
constraints+= [A[I1[0],:]@z>=1]
feasible = cp.Problem(objective,constraints)
feasible.solve(solver='SCS')
S_rec = np.where( z.value>1e-3 )[0]  # note: 'np.where(z.value)[0]' will work if the simplex algorithm is used
print('The original and recovered supports agree:')
print(S)
print(S_rec)

The original and recovered supports agree:
[ 9389 11317 24810 27649]
[ 9389 11317 24810 27649]
